<a href="https://colab.research.google.com/github/shirinsitara/IMLO/blob/last-update/Copy_of_WorkingCopy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
#transform = transforms.ToTensor()
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize all images to 224x224
    transforms.ToTensor(),          # Convert images to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
])

In [4]:
#train_data = datasets.MNIST(root='/cnn_data', train=True, download=True, transform=transform)

In [5]:
from torchvision.datasets import Flowers102

# Load the training data
train_data = Flowers102(root='/flower_data', split='train', download=True, transform=transform)

# Load the validation data
val_data = Flowers102(root='/flower_data', split='val', download=True, transform=transform)

# Load the test data
test_data = Flowers102(root='/flower_data', split='test', download=True, transform=transform)


100%|██████████| 344862509/344862509 [00:16<00:00, 21445021.71it/s]


Extracting /flower_data/flowers-102/102flowers.tgz to /flower_data/flowers-102


100%|██████████| 502/502 [00:00<00:00, 822476.80it/s]


100%|██████████| 14989/14989 [00:00<00:00, 18128149.55it/s]


In [6]:
train_data

Dataset Flowers102
    Number of datapoints: 1020
    Root location: /flower_data
    split=train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [7]:
test_data

Dataset Flowers102
    Number of datapoints: 6149
    Root location: /flower_data
    split=test
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [8]:
val_data

Dataset Flowers102
    Number of datapoints: 1020
    Root location: /flower_data
    split=val
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [9]:
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
valid_loader = DataLoader(val_data, batch_size=10, shuffle=False)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

In [25]:
conv1 = nn.Conv2d(3, 10, 3, 1)
conv2 = nn.Conv2d(10, 32, 3, 2)
conv3 = nn.Conv2d(32, 64, 3, 2)
#conv4 = nn.Conv2d(64, 128, 3, 2)

In [26]:
for i, (X_Train, y_train) in enumerate(train_data):
  break


In [27]:
X_Train.shape

torch.Size([3, 224, 224])

In [28]:
x = X_Train.view(1,3,224,224)

In [29]:
x = F.relu(conv1(x))
x = F.relu(conv2(x))
x = F.max_pool2d(x,1,1)
x = F.relu(conv3(x))
x = F.max_pool2d(x, 1, 1)
#x = F.relu(conv4(x))

In [30]:
x.shape

torch.Size([1, 64, 54, 54])

In [31]:
class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3,10,3,1)
    self.conv2 = nn.Conv2d(10,32,3,2)
    self.conv3 = nn.Conv2d(32, 64, 3, 2)
    #self.conv4 = nn.Conv2d(64, 128, 3, 2)
    # Fully Connected Layer
    self.fc1 = nn.Linear(64*54*54, 1200)
    self.fc2 = nn.Linear(1200, 500)
    self.fc3 = nn.Linear(500, 102)


  def forward(self, X):
    X = F.relu(self.conv1(X))
    #X = F.max_pool2d(X,2,2)

    X = F.relu(self.conv2(X))
    X = F.max_pool2d(X,1,1)

    X = F.relu(self.conv3(X))
    X = F.max_pool2d(X, 1, 1)

    #X = F.relu(self.conv4(X))

    # Re-View to flatten it out
    X = X.view(-1, 64*54*54)

    # Fully Connected Layers
    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X = self.fc3(X)
    return F.log_softmax(X, dim=1)

In [32]:
#torch.manual_seed(41)
model = ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(10, 32, kernel_size=(3, 3), stride=(2, 2))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
  (fc1): Linear(in_features=186624, out_features=1200, bias=True)
  (fc2): Linear(in_features=1200, out_features=500, bias=True)
  (fc3): Linear(in_features=500, out_features=102, bias=True)
)

In [35]:
# Loss Function Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.01)

In [36]:
import time
start_time = time.time()

# Create Variables To Tracks Things
epochs = 50
train_losses = []
test_losses = []
train_correct = []
test_correct = []

# For Loop of Epochs
for i in range(epochs):
  trn_corr = 0
  tst_corr = 0


  # Train
  for b,(X_train, y_train) in enumerate(train_loader):
    b+=1 # start our batches at 1
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    predicted = torch.max(y_pred.data, 1)[1] # add up the number of correct predictions. Indexed off the first point
    batch_corr = (predicted == y_train).sum() # how many we got correct from this batch. True = 1, False=0, sum those up
    trn_corr += batch_corr # keep track as we go along in training.

    # Update our parameters
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


    # Print out some results
    # if b%10 == 0:
    #   print(f'Epoch: {i}  Batch: {b}  Loss: {loss.item()}')

  train_losses.append(loss)
  train_correct.append(trn_corr)


  # Test
  with torch.no_grad(): #No gradient so we don't update our weights and biases with test data
    for b,(X_test, y_test) in enumerate(valid_loader):
      y_val = model(X_test)
      predicted = torch.max(y_val.data, 1)[1] # Adding up correct predictions
      tst_corr += (predicted == y_test).sum() # T=1 F=0 and sum away


  loss = criterion(y_val, y_test)
  test_losses.append(loss)
  test_correct.append(tst_corr)

  train_accuracy = trn_corr / len(train_loader.dataset) * 100
  test_accuracy = tst_corr / len(valid_loader.dataset) * 100

  print(f'Epoch {i+1} - Training Accuracy: {train_accuracy:.2f}% - Validation Accuracy: {test_accuracy:.2f}%')



current_time = time.time()
total = current_time - start_time
print(f'Training Took: {total/60} minutes!')

Epoch 1 - Training Accuracy: 87.16% - Validation Accuracy: 5.59%
Epoch 2 - Training Accuracy: 73.04% - Validation Accuracy: 4.80%
Epoch 3 - Training Accuracy: 89.51% - Validation Accuracy: 3.53%
Epoch 4 - Training Accuracy: 96.86% - Validation Accuracy: 3.92%
Epoch 5 - Training Accuracy: 96.67% - Validation Accuracy: 2.94%
Epoch 6 - Training Accuracy: 89.22% - Validation Accuracy: 2.84%
Epoch 7 - Training Accuracy: 86.67% - Validation Accuracy: 4.02%
Epoch 8 - Training Accuracy: 89.71% - Validation Accuracy: 3.53%
Epoch 9 - Training Accuracy: 96.27% - Validation Accuracy: 2.84%
Epoch 10 - Training Accuracy: 98.24% - Validation Accuracy: 2.94%
Epoch 11 - Training Accuracy: 99.80% - Validation Accuracy: 3.14%
Epoch 12 - Training Accuracy: 100.00% - Validation Accuracy: 2.55%
Epoch 13 - Training Accuracy: 98.73% - Validation Accuracy: 2.55%
Epoch 14 - Training Accuracy: 79.61% - Validation Accuracy: 2.45%
Epoch 15 - Training Accuracy: 89.90% - Validation Accuracy: 2.06%
Epoch 16 - Trainin

KeyboardInterrupt: 

In [ ]:
train_losses = [tl.item() for tl in train_losses]
plt.plot(train_losses, label="Training Loss")
plt.plot(test_losses, label="Validation Loss")
plt.title("Loss at Epoch")
plt.legend()

In [ ]:
plt.plot([t/10 for t in train_correct], label="Training Accuracy")
plt.plot([t/100 for t in test_correct], label="Validation Accuracy")
plt.title("Accuracy at the end of each Epoch")
plt.legend()